# Abrir numpy

In [3]:
import numpy as np

In [4]:
sentences = ['Juan come de la mesa','Pedro come unos tacos','Juan prepara tacos']

In [5]:
from collections import defaultdict

def vocab():
    dicc = defaultdict()
    dicc.default_factory = lambda: len(dicc)
    return dicc

def BoW(corpus,vocab):
    for w in corpus:
        yield[vocab[w_i] for w_i in w.split()]
        
words = vocab()
BagOfWords = list(BoW(sentences,words))

In [6]:
print BagOfWords

[[0, 1, 2, 3, 4], [5, 1, 6, 7], [0, 8, 7]]


In [7]:
print words["Juan"]


0


In [8]:
print words["Pedro"]

5


In [9]:
A = np.zeros((len(words),len(words)))
print A.shape

(9, 9)


# Modelo distribucional simple

In [10]:
for w1 in words:
    for w2 in words:
        cooc = 0
        for context in BagOfWords:
            if words[w1] in context and words[w2] in context and w1 != w2:
                cooc += 1
        A[words[w1],words[w2]] = cooc
    
print A

[[ 0.  1.  1.  1.  1.  0.  0.  1.  1.]
 [ 1.  0.  1.  1.  1.  1.  1.  1.  0.]
 [ 1.  1.  0.  1.  1.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  1.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  1.  1.  0.]
 [ 0.  1.  0.  0.  0.  1.  0.  1.  0.]
 [ 1.  1.  0.  0.  0.  1.  1.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  1.  0.]]


In [11]:
print A[words["Juan"]]

[ 0.  1.  1.  1.  1.  0.  0.  1.  1.]


Ahora, para realizar la reducción en valores singulares, primero usaremos la descomposición en tres matrices distintas. La matriz S que es la diagonal formada por las raíces cuadradas de los valores propios de A. Q y D que son matrices formadas por los vectores propios de AA^t y A^tA, respectivamente. 

In [12]:
Q,S,D = np.linalg.svd(A)
print S

[ 4.64713946  2.18631691  1.77173753  1.68348525  1.3247678   1.          1.
  1.          0.94653421]


In [16]:
k = 2
Q = Q[:,:k]
S = np.diag(S)[:k,:k]
print S

[[ 4.64713946  0.        ]
 [ 0.          2.18631691]]


In [17]:
B = np.zeros((len(words),k))
for i,w in enumerate(A):
    B[i] = np.dot(np.linalg.inv(S),np.dot(Q.T,w))
    
print B

[[-0.4302777  -0.15239713]
 [-0.47969079  0.0898753 ]
 [-0.3437554  -0.3355671 ]
 [-0.3437554  -0.3355671 ]
 [-0.3437554  -0.3355671 ]
 [-0.22087942  0.45364905]
 [-0.22087942  0.45364905]
 [-0.32588726  0.44829623]
 [-0.16271622  0.13534136]]


In [18]:
print B[words["Juan"]]

[-0.4302777  -0.15239713]


In [23]:
def plot_words(Z,ids):
    import matplotlib.pyplot as plt
    r=0
    plt.scatter(Z[:,0],Z[:,1], marker='o', c='blue')
    for label,x,y in zip(ids, Z[:,0], Z[:,1]):
        plt.annotate(label.decode('utf8'), xy=(x,y), xytext=(-1,1), textcoords='offset points', ha='center', va='bottom')
        r+=1
    plt.show()

plot_words(B,words.keys())

In [21]:
def cos(x,y):
    return np.dot(x,y)/( np.linalg.norm(x)*np.linalg.norm(y) )

print cos(A[words["Juan"]],A[words["Pedro"]])

0.471404520791


In [22]:
print cos(A[words["Pedro"]],A[words["mesa"]])

0.288675134595
